<a href="https://colab.research.google.com/github/kiki-glitch/Swa2Text/blob/main/Swa2Tex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pickle
import librosa
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from IPython.display import Audio

import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import * 
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/MyDrive/Swahili/data/train/wav")

## Inputs

In [ ]:
import glob

In [ ]:
rows = []
p_dir = "./"
parent_dir =p_dir +"SWH-05-20101106"
files = os.listdir(parent_dir)
for f in files:
    audio, fs = librosa.load(f"{parent_dir}/{f}")
    filename = f.split('.')[0]
    row = {'filename': filename, 'audio': audio}
    rows.append(row)
rows[:5]

# for filename in glob.glob(os.path.join(p_dir,'*','*.wav')):
#     audio, fs = librosa.load(f"{filename}")
#     filename = f.split('.')[0]
#     row = {'filename': filename, 'audio': audio}
#     rows.append(row)
# rows[:5]

[{'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part28',
  'audio': array([-0.03379978, -0.03822443, -0.03214817, ..., -0.16410704,
         -0.17117754, -0.1036144 ], dtype=float32)},
 {'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part182',
  'audio': array([ 0.002242  ,  0.00087491, -0.00296297, ..., -0.18095487,
         -0.13124813,  0.        ], dtype=float32)},
 {'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part139',
  'audio': array([0.00549206, 0.0058774 , 0.00441616, ..., 0.0360385 , 0.02359483,
         0.00684485], dtype=float32)},
 {'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part180',
  'audio': array([-0.01427129, -0.01713051, -0.0142272 , ..., -0.12489323,
         -0.08557392,  0.        ], dtype=float32)},
 {'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part156',
  'audio': array([-0.0446428 , -0.0511893 , -0.036221  

In [ ]:
len(files)

205

In [ ]:
sample_audios = []
for row in rows:
    audio = row['audio']
    sample_audios.append(audio)
sample_audios[:5]

[array([-0.03379978, -0.03822443, -0.03214817, ..., -0.16410704,
        -0.17117754, -0.1036144 ], dtype=float32),
 array([ 0.002242  ,  0.00087491, -0.00296297, ..., -0.18095487,
        -0.13124813,  0.        ], dtype=float32),
 array([0.00549206, 0.0058774 , 0.00441616, ..., 0.0360385 , 0.02359483,
        0.00684485], dtype=float32),
 array([-0.01427129, -0.01713051, -0.0142272 , ..., -0.12489323,
        -0.08557392,  0.        ], dtype=float32),
 array([-0.0446428 , -0.0511893 , -0.036221  , ...,  0.15283288,
         0.10397086,  0.        ], dtype=float32)]

In [ ]:
meta_df = pd.read_csv('/content/drive/MyDrive/Swahili/metadata.csv')

In [ ]:
meta_df.head()

,filename,transcription,filepath,sample_rate,duration
0,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,rais wa tanzania jakaya mrisho kikwete,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.14
1,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,yanayo andaliwa nami pendo pondo idhaa ya kisw...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.10
2,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,inayokutangazia moja kwa moja kutoka jijini da...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.65
3,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,juma hili bara la afrika limeshuhudia raia wa ...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.90
4,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,wakipiga kura ya maoni ilikufanya mabadiliko ya,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,2.94


In [ ]:
txts = []
for row in rows:
    filename = row['filename']
    filter = meta_df[meta_df['filename'] == filename]
    txt = filter[['transcription']].values
    txts.append(txt)

txts[:5]

[array([['kutoka viwanja vya uhuru hapa nchini tanzania']], dtype=object),
 array([['watanzania watarajie nini wapiga kura wana matarajio makubwa sana na wabunge wao']],
       dtype=object),
 array([['ndiyo imekuwa dosari kubwa inaotajwa kutokea']], dtype=object),
 array([['kuwa tuna watuma yaani kile tunachotaka wakifanye']],
       dtype=object),
 array([['hadi sasa inaridhisha <UNK>']], dtype=object)]

In [ ]:
txts = np.array(txts).reshape(-1)

In [ ]:
txts[:5]

array(['kutoka viwanja vya uhuru hapa nchini tanzania',
       'watanzania watarajie nini wapiga kura wana matarajio makubwa sana na wabunge wao',
       'ndiyo imekuwa dosari kubwa inaotajwa kutokea',
       'kuwa tuna watuma yaani kile tunachotaka wakifanye',
       'hadi sasa inaridhisha <UNK>'], dtype=object)

In [ ]:
clean_txts = []
alphabets = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'.split()
for txt in txts:
    clean_txt = []
    for c in txt:
        if c not in alphabets and c != ' ':
            continue
        clean_txt.append(c)
    clean_txt = ''.join(clean_txt)
    clean_txts.append(clean_txt)

In [ ]:
clean_txts[:5]

['kutoka viwanja vya uhuru hapa nchini tanzania',
 'watanzania watarajie nini wapiga kura wana matarajio makubwa sana na wabunge wao',
 'ndiyo imekuwa dosari kubwa inaotajwa kutokea',
 'kuwa tuna watuma yaani kile tunachotaka wakifanye',
 'hadi sasa inaridhisha ']

In [ ]:
'' in clean_txts

True

In [ ]:
df = pd.DataFrame(clean_txts)
df.columns = ['texts']
df.head()

,texts
0,kutoka viwanja vya uhuru hapa nchini tanzania
1,watanzania watarajie nini wapiga kura wana mat...
2,ndiyo imekuwa dosari kubwa inaotajwa kutokea
3,kuwa tuna watuma yaani kile tunachotaka wakifanye
4,hadi sasa inaridhisha


In [ ]:
idxs = df[df['texts'] == ''].index
idxs

Int64Index([49, 91, 123], dtype='int64')

In [ ]:
del clean_txts[idxs[-1]]
del clean_txts[idxs[-2]]
del clean_txts[idxs[-3]]

In [ ]:
'' in clean_txts

False

In [ ]:
del sample_audios[idxs[-1]]
del sample_audios[idxs[-2]]
del sample_audios[idxs[-3]]

## Tokenizer

In [ ]:
def character_dict():
    alphabet = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'
    supported = alphabet.split()

    char_map = {}
    char_map[""] = 0
    char_map["<SPACE>"] = 1
    idx = 2
    for c in supported:
        char_map[c] = idx
        idx += 1
    index_map = {v: k for k, v in char_map.items()}
    return char_map, index_map

In [ ]:
char_map, index_map = character_dict()

In [ ]:
char_map

{'': 0,
 '<SPACE>': 1,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'j': 11,
 'k': 12,
 'l': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'p': 17,
 'q': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'x': 25,
 'y': 26,
 'z': 27}

In [ ]:
def text_to_int_sequence(text):
    """ Convert text to an integer sequence """
    int_sequence = []
    for c in text:
        if c == ' ':
            ch = char_map['<SPACE>']
        elif c in alphabets:
            ch = char_map[c]
        else:
            print(c)
            print('character not found')
            break
        int_sequence.append(ch)
    return np.array(int_sequence)

In [ ]:
def int_sequence_to_text(int_sequence):
    """ Convert an integer sequence to text """
    textch = []
    for c in int_sequence:
        ch = index_map[c]
        textch.append(ch)
    text = ''.join(textch)
    text = text.replace('<SPACE>', ' ')
    return text

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, audios, texts, batch_size=32):
        self.audios = audios
        self.texts = texts
        self.batch_size = batch_size
        self.steps = int(len(self.audios) // self.batch_size)
        self.on_epoch_end()

    def __len__(self):
        return self.steps

    def on_epoch_end(self):
        self.indexes = np.arange(self.steps*self.batch_size)
        # np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        indexes = self.indexes[int(index*self.batch_size):int((index+1)*self.batch_size)]
    
        batch_audios = [self.audios[int(i)] for i in indexes]
        batch_texts = [self.texts[int(i)] for i in indexes]
        
        return  self.data_generation(batch_audios, batch_texts)
    
    def data_generation(self, batch_audios, batch_texts):

        longest_audio = max([len(i) for i in batch_audios])
        longest_txt = max([len(i) for i in batch_texts])

        audios          = np.zeros([int(self.batch_size), longest_audio], dtype="float32")
        txts            = np.zeros([int(self.batch_size), longest_txt], dtype="int64")
        audio_length    = np.zeros([int(self.batch_size)], dtype="int64")
        txt_length      = np.zeros([int(self.batch_size)], dtype="int64")

        i = 0
        for audio, txt in zip(batch_audios, batch_texts):

            txt_len = len(txt)

            txt = text_to_int_sequence(txt)
       
            txts[i,: txt_len] = txt

            audio_len = len(audio)

            audios[i, :audio_len] = audio

            audio_length[i] = audio_len
            txt_length[i] = txt_len

            i+=1          
            
        outputs = {'ctc': np.zeros([self.batch_size])}
        inputs = {
                    'the_input':    tf.convert_to_tensor(audios), 
                    'the_labels':   tf.convert_to_tensor(txts), 
                    'input_length': tf.convert_to_tensor(audio_length), 
                    'label_length': tf.convert_to_tensor(txt_length)
                }
        return (inputs, outputs)


In [ ]:
dg = DataGenerator(sample_audios, clean_txts)

In [ ]:
len(dg)

6

In [ ]:
dg[5]

({'the_input': <tf.Tensor: shape=(32, 135387), dtype=float32, numpy=
  array([[-0.00909726, -0.01020271, -0.00806536, ...,  0.        ,
           0.        ,  0.        ],
         [-0.00962744, -0.00908791, -0.0040448 , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.0063101 ,  0.00703578,  0.0055557 , ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [-0.00993562, -0.01121048, -0.00931332, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.0015532 ,  0.00214485,  0.0019096 , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.00304882,  0.00447894,  0.0048337 , ...,  0.        ,
           0.        ,  0.        ]], dtype=float32)>,
  'the_labels': <tf.Tensor: shape=(32, 112), dtype=int64, numpy=
  array([[11,  6, 11, ...,  0,  0,  0],
         [ 9,  2, 12, ...,  0,  0,  0],
         [10,  5,  9, ...,  0,  0,  0],
         ...,
         [13,  2,  1, ...,  0,  0,  0],
         [15,  2,  1, ...,  0,  0,  

In [ ]:
batch1 = dg[0][0]


In [ ]:
batch1['the_labels']

<tf.Tensor: shape=(32, 116), dtype=int64, numpy=
array([[12, 22, 21, ...,  0,  0,  0],
       [24,  2, 21, ...,  0,  0,  0],
       [15,  5, 10, ...,  0,  0,  0],
       ...,
       [12,  2, 27, ...,  0,  0,  0],
       [11,  2, 12, ...,  0,  0,  0],
       [12, 24,  2, ...,  0,  0,  0]])>

In [ ]:
batch1['the_input']

<tf.Tensor: shape=(32, 123700), dtype=float32, numpy=
array([[-0.03379978, -0.03822443, -0.03214817, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.002242  ,  0.00087491, -0.00296297, ..., -0.18095487,
        -0.13124813,  0.        ],
       [ 0.00549206,  0.0058774 ,  0.00441616, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.03675308, -0.04571614, -0.03824458, ...,  0.        ,
         0.        ,  0.        ],
       [-0.00065874, -0.00252594, -0.00411112, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00060241, -0.00034816, -0.00233458, ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)>

### ctc


In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [ ]:
def input_lengths_lambda_func(args):
    input_length = args
    return tf.cast(tf.math.floor(input_length/hop_size)-1, dtype="float32")

In [ ]:
def add_ctc_loss(model_builder):
    the_labels      = Input(name='the_labels',      shape=(None,), dtype='float32')
    input_lengths   = Input(name='input_length',    shape=(1,), dtype='float32')
    label_lengths   = Input(name='label_length',    shape=(1,), dtype='float32')

    input_lengths2 = Lambda(input_lengths_lambda_func)(input_lengths)
    if model_builder.output_length:
         output_lengths  = Lambda(model_builder.output_length)(input_lengths2)
    else:
         output_lengths  = input_lengths2
    
    # CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([model_builder.output, the_labels, output_lengths, label_lengths])
    model = Model( inputs=[model_builder.input, the_labels, input_lengths, label_lengths],  outputs=loss_out)
    return model

## LogMelSpectrogram

In [ ]:
class LogMelSpectrogram(tf.keras.layers.Layer):
    """Compute log-magnitude mel-scaled spectrograms."""

    def __init__(self, sample_rate, fft_size, hop_size, n_mels,
                 f_min=0.0, f_max=None, **kwargs):
        super(LogMelSpectrogram, self).__init__(**kwargs)
        self.sample_rate = sample_rate
        self.fft_size = fft_size
        self.hop_size = hop_size
        self.n_mels = n_mels
        self.f_min = f_min
        self.f_max = f_max if f_max else sample_rate / 2
        self.mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
            num_mel_bins=self.n_mels,
            num_spectrogram_bins=fft_size // 2 + 1,
            sample_rate=self.sample_rate,
            lower_edge_hertz=self.f_min,
            upper_edge_hertz=self.f_max)

    def build(self, input_shape):
        self.non_trainable_weights.append(self.mel_filterbank)
        super(LogMelSpectrogram, self).build(input_shape)

    def call(self, waveforms):
        """Forward pass.
        Parameters
        ----------
        waveforms : tf.Tensor, shape = (None, n_samples)
            A Batch of mono waveforms.
        Returns
        -------
        log_mel_spectrograms : (tf.Tensor), shape = (None, time, freq, ch)
            The corresponding batch of log-mel-spectrograms
        """
        def _tf_log10(x):
            numerator = tf.math.log(x)
            denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
            return numerator / denominator

        def power_to_db(magnitude, amin=1e-16, top_db=80.0):
            """
            https://librosa.github.io/librosa/generated/librosa.core.power_to_db.html
            """
            ref_value = tf.reduce_max(magnitude)
            log_spec = 10.0 * _tf_log10(tf.maximum(amin, magnitude))
            log_spec -= 10.0 * _tf_log10(tf.maximum(amin, ref_value))
            log_spec = tf.maximum(log_spec, tf.reduce_max(log_spec) - top_db)

            return log_spec

        spectrograms = tf.signal.stft(waveforms,
                                      frame_length=self.fft_size,
                                      frame_step=self.hop_size,
                                      pad_end=False)

        magnitude_spectrograms = tf.abs(spectrograms)

        mel_spectrograms = tf.matmul(tf.square(magnitude_spectrograms),
                                     self.mel_filterbank)

        log_mel_spectrograms = power_to_db(mel_spectrograms)

        # add channel dimension
        log_mel_spectrograms = tf.expand_dims(log_mel_spectrograms, 3)

        return log_mel_spectrograms

    def get_config(self):
        config = {
            'fft_size': self.fft_size,
            'hop_size': self.hop_size,
            'n_mels': self.n_mels,
            'sample_rate': self.sample_rate,
            'f_min': self.f_min,
            'f_max': self.f_max,
        }
        config.update(super(LogMelSpectrogram, self).get_config())

        return config

## Models

In [ ]:
def preprocessin_model(sample_rate, fft_size, frame_step, n_mels, mfcc=False):

    input_data = Input(name='input', shape=(None,), dtype="float32")
    featLayer = LogMelSpectrogram(
        fft_size=fft_size,
        hop_size=frame_step,
        n_mels=n_mels,
        
        sample_rate=sample_rate,
        f_min=0.0,
        
        f_max=int(sample_rate / 2)
    )(input_data)
    
    x = BatchNormalization()(featLayer)
    model = Model(inputs=input_data, outputs=x, name="preprocessin_model")

    return model

In [ ]:
def simple_rnn_model(input_dim, output_dim=224):

    input_data = Input(name='the_input', shape=(None, input_dim))
    simp_rnn = GRU(output_dim, return_sequences=True,
                   implementation=2, name='rnn')(input_data)
    y_pred = Activation('softmax', name='softmax')(simp_rnn)
    model = Model(inputs=input_data, outputs=y_pred, name="simple_rnn_model")
    model.output_length = lambda x: x
    return model


In [ ]:
def BidirectionalRNN(input_dim, rnn_layers=2, units=400, drop_out=0.5, act='tanh', output_dim=224):

    input_data = Input(name='the_input', shape=(
        None, input_dim))

    x = Bidirectional(LSTM(units,  activation=act,
                      return_sequences=True, implementation=2))(input_data)
    x = BatchNormalization()(x)
    x = Dropout(drop_out)(x)

    for i in range(rnn_layers - 2):
        x = Bidirectional(
            LSTM(units, activation=act, return_sequences=True))(x)
        x = BatchNormalization()(x)
        x = Dropout(drop_out)(x)

    x = Bidirectional(LSTM(units,  activation=act,
                      return_sequences=True, implementation=2))(x)
    x = BatchNormalization()(x)
    x = Dropout(drop_out)(x)

    time_dense = TimeDistributed(Dense(output_dim))(x)

    y_pred = Activation('softmax', name='softmax')(time_dense)

    model = Model(inputs=input_data, outputs=y_pred, name="BidirectionalRNN")

    return model

In [ ]:
def conv_rnn(n_mels, output_dim=224, rnn_layers=4, units=400, drop_out=0.5, act='tanh'):

    input_data = Input(name='the_input', shape=(None, n_mels, 1))

    y = Conv2D(32, (3, 3), padding='same')(input_data)  # was 32
    y = Activation('relu')(y)
    y = BatchNormalization()(y)

    x = MaxPooling2D((1, 2))(y)

    x = Conv2D(64, (3, 3), padding='same')(y)  # was 32
    x = Activation('relu')(x)
    y = BatchNormalization()(y)

    x = MaxPooling2D((1, 2))(x)

    x = Conv2D(128, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    y = BatchNormalization()(y)

    x = MaxPooling2D((1, 2))(x)

    x = Dense(128)(x)
    x = Dense(64)(x)
    x = Dense(32)(x)

    x = Reshape((-1, x.shape[-1] * x.shape[-2]))(x)

    for i in range(rnn_layers):
        x = Bidirectional(
            LSTM(units, activation=act, return_sequences=True))(x)
        x = BatchNormalization()(x)
        x = Dropout(drop_out)(x)

    bn_rnn = BatchNormalization()(x)

    time_dense = TimeDistributed(Dense(output_dim))(bn_rnn)

    y_pred = Activation('softmax', name='softmax')(time_dense)

    model = Model(inputs=input_data, outputs=y_pred, name="custom_model")

    return model

In [ ]:
def train(model_builder, 
          data_gen,
          epochs, 
          verbose=1,
          optimizer=SGD(learning_rate=0.002, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
          ):    
              
    model = add_ctc_loss(model_builder)

    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    print(model.summary())

    # add checkpointer
    checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/Swahili/data/train/Model/cnn_rnn3.h5', verbose=1)
    early_stopping = EarlyStopping( monitor="val_loss", patience=10, restore_best_weights=True)

    hist = model.fit_generator(generator=data_gen,
                               epochs=epochs,
                               callbacks=[checkpointer, early_stopping], 
                               verbose=verbose, 
                               use_multiprocessing=False)
    
    # save model loss
    with open('/content/drive/MyDrive/Swahili/data/train/Model/cnn_rnn.pickle', 'wb') as f:
        pickle.dump(hist.history, f)

### Trainig

In [ ]:

sample_rate = 22050
fft_size = 1024
frame_step = 512
n_mels = 128

batch_size = 32
epochs = 100
data_len = len(clean_txts)
output_dim = len(char_map) + 2


In [ ]:
dg = DataGenerator(sample_audios, clean_txts, batch_size)

In [ ]:
preprocess_model = preprocessin_model(sample_rate, fft_size, frame_step, n_mels)
preprocess_model.summary()


Model: "preprocessin_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, None)]            0         
                                                                 
 log_mel_spectrogram_4 (LogM  (None, None, 128, 1)     0         
 elSpectrogram)                                                  
                                                                 
 batch_normalization_36 (Bat  (None, None, 128, 1)     4         
 chNormalization)                                                
                                                                 
Total params: 4
Trainable params: 2
Non-trainable params: 2
_________________________________________________________________


In [ ]:
speech_model = conv_rnn(n_mels, output_dim = output_dim)
speech_model.summary()


Model: "custom_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None, 128, 1)]    0         
                                                                 
 conv2d_12 (Conv2D)          (None, None, 128, 32)     320       
                                                                 
 activation_12 (Activation)  (None, None, 128, 32)     0         
                                                                 
 batch_normalization_37 (Bat  (None, None, 128, 32)    128       
 chNormalization)                                                
                                                                 
 conv2d_13 (Conv2D)          (None, None, 128, 64)     18496     
                                                                 
 activation_13 (Activation)  (None, None, 128, 64)     0         
                                                      

In [ ]:
def build_model(output_dim, custom_model, preprocess_model, calc=None):

    input_audios = Input(name='the_input', shape=(None,))
    pre = preprocess_model(input_audios)
    pre = tf.squeeze(pre, [3])

    y_pred = custom_model(pre)
    model = Model(inputs=input_audios, outputs=y_pred, name="model_builder")
    model.output_length = calc

    return model

In [ ]:
model = build_model(output_dim, speech_model, preprocess_model)
model.summary()


Model: "model_builder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None)]            0         
                                                                 
 preprocessin_model (Functio  (None, None, 128, 1)     4         
 nal)                                                            
                                                                 
 tf.compat.v1.squeeze_4 (TFO  (None, None, 128)        0         
 pLambda)                                                        
                                                                 
 custom_model (Functional)   (None, None, 30)          16249278  
                                                                 
Total params: 16,249,282
Trainable params: 16,241,216
Non-trainable params: 8,066
_________________________________________________________________


In [ ]:
import h5py
hf = h5py.File('/content/drive/MyDrive/Swahili/data/train/Model/cnn_rnn3.h5', 'w')
hf.close()

In [ ]:
model.load_weights('/content/drive/MyDrive/Swahili/data/train/Model/cnn_rnn3.h5', by_name = True, skip_mismatch = True)

In [ ]:
 !pip install mlflow --quiet

     |████████████████████████████████| 17.0 MB 14.5 MB/s 
     |████████████████████████████████| 77 kB 7.4 MB/s 
     |████████████████████████████████| 209 kB 63.6 MB/s 
     |████████████████████████████████| 182 kB 70.7 MB/s 
     |████████████████████████████████| 147 kB 56.4 MB/s 
     |████████████████████████████████| 79 kB 9.2 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 140 kB 51.3 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 


In [ ]:
 import mlflow

In [ ]:
# mlflow.set_experiment('Speech Model-RNN-baseline')
# mlflow.tensorflow.autolog()
hop_size = 512
train(model, dg, epochs=100)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 the_input (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 preprocessin_model (Functional  (None, None, 128, 1  4          ['the_input[0][0]']              
 )                              )                                                                 
                                                                                                  
 tf.compat.v1.squeeze_4 (TFOpLa  (None, None, 128)   0           ['preprocessin_model[0][0]']     
 mbda)                                                                                            
                                                                                              

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


6/6 [==============================] - ETA: 0s - loss: 286.8180
Epoch 1: saving model to /content/drive/MyDrive/Swahili/data/train/Model/cnn_rnn3.h5


6/6 [==============================] - 46s 2s/step - loss: 286.8180
Epoch 2/100
6/6 [==============================] - ETA: 0s - loss: 195.5413
Epoch 2: saving model to /content/drive/MyDrive/Swahili/data/train/Model/cnn_rnn3.h5


6/6 [==============================] - 5s 821ms/step - loss: 195.5413
Epoch 3/100
6/6 [==============================] - ETA: 0s - loss: 183.9459
Epoch 3: saving model to /content/drive/MyDrive/Swahili/data/train/Model/cnn_rnn3.h5


...

6/6 [==============================] - 4s 700ms/step - loss: 42.0507
Epoch 97/100
6/6 [==============================] - ETA: 0s - loss: 41.8700
Epoch 97: saving model to /content/drive/MyDrive/Swahili/data/train/Model/cnn_rnn3.h5


6/6 [==============================] - 4s 661ms/step - loss: 41.8700
Epoch 98/100
6/6 [==============================] - ETA: 0s - loss: 41.3852
Epoch 98: saving model to /content/drive/MyDrive/Swahili/data/train/Model/cnn_rnn3.h5


6/6 [==============================] - 4s 662ms/step - loss: 41.3852
Epoch 99/100
6/6 [==============================] - ETA: 0s - loss: 40.5622
Epoch 99: saving model to /content/drive/MyDrive/Swahili/data/train/Model/cnn_rnn3.h5


6/6 [==============================] - 4s 667ms/step - loss: 40.5622
Epoch 100/100
6/6 [==============================] - ETA: 0s - loss: 40.7583
Epoch 100: saving model to /content/drive/MyDrive/Swahili/data/train/Model/cnn_rnn3.h5


6/6 [==============================] - 4s 679ms/step - loss: 40.7583


In [ ]:
!pip install jiwer --quiet

     |████████████████████████████████| 1.4 MB 20.5 MB/s 
     |████████████████████████████████| 2.2 MB 52.0 MB/s 


In [ ]:
from jiwer import wer

In [ ]:
def predict(data_gen,  num_elem=1, index=0):
    
    pred_data = data_gen.__getitem__(index)

    pred_audios = pred_data[0]["the_input"]
    pred_labels = pred_data[0]["the_labels"]
    
    y_pred = model.predict(pred_audios)

    input_shape = tf.keras.backend.shape(y_pred)
    input_length = tf.ones(shape=input_shape[0]) * tf.keras.backend.cast(input_shape[1], 'float32')
    prediction = tf.keras.backend.ctc_decode(y_pred, input_length, greedy=False)[0][0]
    

    for i in range(0, num_elem):  # only on clean data
        
        pred = K.eval(prediction[i]).flatten().tolist()
        pred = [i for i in pred if i != -1]



        ground_truth = int_sequence_to_text(pred_labels[i].numpy())
        hypothesis   = ''.join(int_sequence_to_text(pred))
        # error        = wer(ground_truth, hypothesis)
                
        print('-'*48 + ' ' + str(i) + ' ' + '-'*48)
        print('True transcription:\n' + '\n' + ground_truth)
        print('-'*100)
        print('Predicted transcription:\n' + '\n' + hypothesis)
        # print('-'*100)
        # print('Word Error Rate:' + str(error))
        print('\n')

In [ ]:
rows1 = []
parent_dir1 = "./SWH-05-20101107"
files1 = os.listdir(parent_dir1)
for f in files1:
    audio, fs = librosa.load(f"{parent_dir1}/{f}")
    filename = f.split('.')[0]
    row = {'filename': filename, 'audio': audio}
    rows1.append(row)
rows1[:5]

[{'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part33',
  'audio': array([0.00104321, 0.00165943, 0.00152614, ..., 0.03417829, 0.02824923,
         0.01558879], dtype=float32)},
 {'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part170',
  'audio': array([ 0.01966014,  0.00921181, -0.00616249, ...,  0.06976125,
          0.05451657,  0.        ], dtype=float32)},
 {'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part158',
  'audio': array([-0.00656853, -0.00695729, -0.00374735, ..., -0.10506113,
         -0.12291247,  0.        ], dtype=float32)},
 {'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part3',
  'audio': array([ 0.0008225 ,  0.00121391,  0.00155496, ..., -0.11234738,
         -0.11032466, -0.06956042], dtype=float32)},
 {'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part55',
  'audio': array([0.01748345, 0.01829427, 0.01416619, ...,

In [ ]:
sample_audios1 = []
for row in rows1:
    audio = row['audio']
    sample_audios1.append(audio)
sample_audios1[:5]

[array([0.00104321, 0.00165943, 0.00152614, ..., 0.03417829, 0.02824923,
        0.01558879], dtype=float32),
 array([ 0.01966014,  0.00921181, -0.00616249, ...,  0.06976125,
         0.05451657,  0.        ], dtype=float32),
 array([-0.00656853, -0.00695729, -0.00374735, ..., -0.10506113,
        -0.12291247,  0.        ], dtype=float32),
 array([ 0.0008225 ,  0.00121391,  0.00155496, ..., -0.11234738,
        -0.11032466, -0.06956042], dtype=float32),
 array([0.01748345, 0.01829427, 0.01416619, ..., 0.06764735, 0.07200358,
        0.04313861], dtype=float32)]

In [ ]:
meta_df = pd.read_csv('/content/drive/MyDrive/Swahili/metadata.csv')

In [ ]:
txts1 = []
for row in rows1:
    filename = row['filename']
    filter = meta_df[meta_df['filename'] == filename]
    txt = filter[['transcription']].values
    txts1.append(txt)

txts1[:5]

[array([['bi ingabire elize na diude hakizimana <music>']], dtype=object),
 array([['tuna maziwa tuna misitu tuna milima']], dtype=object),
 array([['kwa hivyo mama mwenye mtoto kukaa kituoni wakati anaona hawa wana pita pita']],
       dtype=object),
 array([['habari za asubuhi ni jumapili ya tarehe saba ya mwezi novemba']],
       dtype=object),
 array([['kaa tayari kusikiliza makala haya']], dtype=object)]

In [ ]:
txts1 = np.array(txts1).reshape(-1)

In [ ]:
txts1[:5]

array(['bi ingabire elize na diude hakizimana <music>',
       'tuna maziwa tuna misitu tuna milima',
       'kwa hivyo mama mwenye mtoto kukaa kituoni wakati anaona hawa wana pita pita',
       'habari za asubuhi ni jumapili ya tarehe saba ya mwezi novemba',
       'kaa tayari kusikiliza makala haya'], dtype=object)

In [ ]:
clean_txts1 = []
alphabets = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'.split()
for txt in txts1:
    clean_txt = []
    for c in txt:
        if c not in alphabets and c != ' ':
            continue
        clean_txt.append(c)
    clean_txt = ''.join(clean_txt)
    clean_txts1.append(clean_txt)

In [ ]:
clean_txts1[:5]

['bi ingabire elize na diude hakizimana music',
 'tuna maziwa tuna misitu tuna milima',
 'kwa hivyo mama mwenye mtoto kukaa kituoni wakati anaona hawa wana pita pita',
 'habari za asubuhi ni jumapili ya tarehe saba ya mwezi novemba',
 'kaa tayari kusikiliza makala haya']

In [ ]:
'' in clean_txts1

False

In [ ]:
dg1 = DataGenerator(sample_audios1[:5], clean_txts1[:5], 5)

In [ ]:
predict(dg1, 4)

1/1 [==============================] - 3s 3s/step
------------------------------------------------ 0 ------------------------------------------------
True transcription:

bi ingabire elize na diude hakizimana music
----------------------------------------------------------------------------------------------------
Predicted transcription:

di i nalie hliliz i na jiegtenea kizi manaoaoa a a a a a


------------------------------------------------ 1 ------------------------------------------------
True transcription:

tuna maziwa tuna misitu tuna milima
----------------------------------------------------------------------------------------------------
Predicted transcription:

tunamazi wa kunamishi k kunandilimaoaoa a a a a a


------------------------------------------------ 2 ------------------------------------------------
True transcription:

kwa hivyo mama mwenye mtoto kukaa kituoni wakati anaona hawa wana pita pita
------------------------------------------------------------------

In [ ]:
# Publish Model

In [ ]:
# !sudo apt-get install git-lfs
!pip install transformers==4.23.1
import transformers
!transformers-cli login
!transformers-cli repo ASR_model
!git clone https://huggingface.co/Kiki2022/ASR_model
!cd ASR_model
!git lfs install
# git clone https://huggingface.co/Kiki2022/ASR_model
# sudo apt-get install git-lfs
# cd Swa2Text
# git lfs install
# git config --global user.email “alex.wamai”@strathmore.edu
# git config --global user.name “Alex Wamai”

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/bin/transformers-cli", line 5, in <module>
    from transformers.commands.transformers_cli import main
  File "/usr/local/lib/python3.7/dist-packages/transformers/commands/transformers_cli.py", line 24, in <module>
    from .pt_to_tf import PTtoTFCommand
  File "/usr/local/lib/python3.7/dist-packages/transformers/commands/pt_to_tf.py", line 21, in <module>
    from datasets import load_dataset
ModuleNotFoundError: No module named 'datasets'
Traceback (most recent call last):
  File "/usr/local/bin/transformers-cli", line 5, in <module>
    from transformers.commands.transformers_cli import main
  File "/usr/local/lib/python3.7/dist-packages/transformers/commands/transformers_cli.py", line 24, in <module>
    from .pt_to_tf import PTtoTFCommand
  File "/usr/local/lib/python3.7/dist-packages/transformers/commands/pt_to_tf.py", line 21, i